In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
import gc
import matplotlib.pyplot as plt

import sys
sys.path.append( '../src')

from funcs import *

In [ ]:
prop_2016 = pd.read_csv('../Data/01_raw/properties_2016.csv', low_memory=False)
sample = pd.read_csv('../Data/01_raw/sample_submission.csv', low_memory=False)
train_2016 = pd.read_csv('../Data/01_raw/train_2016.csv', low_memory=False)
pd.set_option('display.max_columns', None)

## Machine Learning Model

In [4]:
x_train = df_ml.drop(['parcelid', 'logerror'], axis=1)
y_train = df_ml['logerror'].values

NameError: name 'df_ml' is not defined

In [6]:
train_columns = x_train.columns

NameError: name 'x_train' is not defined

In [14]:
# division du dataset entre sets de validation, d'entraînement et de test
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

In [15]:
print('Building DMatrix...')

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

Building DMatrix...


In [16]:
del x_train, x_valid; gc.collect()

10704

In [17]:
# Training

params = {}
params['eta'] = 0.02
params['objective'] = 'reg:linear'
params['eval_metric'] = 'mae'
params['max_depth'] = 4
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)

[18:03:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:03:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mae:0.48807	valid-mae:0.48109
[10]	train-mae:0.40226	valid-mae:0.39514
[20]	train-mae:0.33277	valid-mae:0.32555
[30]	train-mae:0.27665	valid-mae:0.26951
[40]	train-mae:0.23150	valid-mae:0.22453
[50]	train-mae:0.19528	valid-mae:0.18856
[60]	train-mae:0.16639	valid-mae:0.15992
[70]	train-mae:0.14344	valid-mae:0.13724
[80]	train-mae:0.12536	valid-mae:0.11944
[90]	train-mae:0.11123	valid-mae:0.10557
[100]	train-mae:0.10031	valid-

In [18]:
del d_train, d_valid

In [19]:
sample = sample.rename({'ParcelId' : 'parcelid'}, axis = 1)
df_test = sample.merge(prop_2016, on='parcelid', how='left')

In [20]:
del prop_2016; gc.collect()

97

In [21]:
props, NAlist = reduce_mem_usage(df_test)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 1480.4008865356445  MB
******************************
Column:  parcelid
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  201610
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  201611
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  201612
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  201710
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  201711
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  201712
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  airconditioningtypeid
dtype before:  float64
dtype after:  

In [22]:
x_test = df_test[train_columns]

In [23]:
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

In [24]:
del df_test, sample; gc.collect()

45

In [25]:
d_test = xgb.DMatrix(x_test)

In [26]:
del x_test; gc.collect()

39

In [27]:
p_test = clf.predict(d_test)

In [28]:
del d_test; gc.collect()

30

In [29]:
sub = pd.read_csv('../Data/01_raw/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

In [30]:
sub

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,-0.278622,-0.278622,-0.278622,-0.278622,-0.278622,-0.278622
1,10759547,-0.278622,-0.278622,-0.278622,-0.278622,-0.278622,-0.278622
2,10843547,0.084676,0.084676,0.084676,0.084676,0.084676,0.084676
3,10859147,-0.156125,-0.156125,-0.156125,-0.156125,-0.156125,-0.156125
4,10879947,-0.069495,-0.069495,-0.069495,-0.069495,-0.069495,-0.069495
...,...,...,...,...,...,...,...
2985212,168176230,1.103344,1.103344,1.103344,1.103344,1.103344,1.103344
2985213,14273630,1.103344,1.103344,1.103344,1.103344,1.103344,1.103344
2985214,168040630,1.103344,1.103344,1.103344,1.103344,1.103344,1.103344
2985215,168040830,1.103344,1.103344,1.103344,1.103344,1.103344,1.103344


In [31]:
print('Writing csv ...')
sub.to_csv('xgb_starter.csv', index=False, float_format='%.4f')

Writing csv ...
